In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.transform import Rotation
from tqdm.notebook import tqdm

In [ ]:
folder = r"J:\Alja Podgornik\Multimaze arena\Cohort 1_June 2020\all_videos\processed"

# Load the combined tracking data if exists
if os.path.exists(os.path.join(folder, 'combined_tracking.h5')):
    print(os.path.join(folder, 'combined_tracking.h5'), 'found.', 'Loading..')
    df = pd.read_hdf(os.path.join(folder, 'combined_tracking.h5'))
    print('Loaded to the memory')
else: # Otherwise create and save the combined tracking data
    print(os.path.join(folder, 'combined_tracking.h5'), 'not found.', 'Creating..')
    data = {}
    for file in tqdm(os.listdir(folder)):
        if 'DLC' in file and file.endswith('.h5'):
            # Prepare group, animal, day information
            filename = file[:file.find('DLC')]
            splitted_name = filename.rsplit(sep='_')
            if len(splitted_name) == 2:
                animal = splitted_name[0]
                day = int(splitted_name[1][-1])
                if animal.startswith('chr'):
                    group = 'chr'
                elif animal.startswith('hr'):
                    group = 'hr'
                elif animal.startswith('ctrl'):
                    group = 'ctrl'

            # Read in the DLC tracking data and append group, animal, day information
            tracking = pd.read_hdf(os.path.join(folder, file))
            tracking = tracking.droplevel(level=0, axis=1)
            tracking['group'] = group
            tracking['animal'] = animal
            tracking['day'] = day
            # Append into the dictionary to concatenate later
            data[file] = tracking

    df = pd.concat(data.values()).reset_index().rename({'index':'frame'}, axis='columns')
    # Change dtypes for compactness
    group_dtype = pd.CategoricalDtype(categories=['chr', 'ctrl', 'hr'], ordered=True)
    dtypes = {'group':group_dtype, 'animal':'category', 'day':'category'}
    df.astype(dtypes)
    # Write to disk
    df = df.set_index(['group', 'animal', 'day', 'frame']).sort_index()
    df.to_hdf(os.path.join(folder, 'combined_tracking.h5'), key='combined_tracking')
    print(os.path.join(folder, 'combined_tracking.h5'), 'created and saved.')

In [ ]:
original_data = df.copy(deep=True)

# Filtering out bad tracking data
df = df.drop(columns=['paw_f_left', 'paw_f_right', 'paw_h_left', 'paw_h_right', 'tail_tip', 'tail_mid'], level=0)
high_likelihood = df.loc[:, df.columns.get_level_values(1) == 'likelihood'] > 0.9
df = df.loc[high_likelihood.all(axis=1), :]
# df = df.drop(columns='likelihood', level=1)

In [ ]:
# Centering the mouse to the origin
center = df['center']
unique_bodyparts = df.columns.get_level_values(0).unique()
unique_axes = df.columns.get_level_values(1).unique()
xy = unique_axes[:-1]

for bodypart in tqdm(unique_bodyparts):
    for axis in xy:
        df.loc[:, (bodypart, axis)] = df.loc[:, (bodypart, axis)] - center.loc[:, axis]

In [ ]:
# Rotating the mouse
rotated = pd.DataFrame(index=df.index, columns=df.columns)
angles = np.arctan2(df[('tail_base', 'y')], df['tail_base', 'x'])

for bodypart in tqdm(unique_bodyparts):
    bp_df = df.loc[:, bodypart]
    bp_x = bp_df['x']
    bp_y = bp_df['y']
    bp_likelihood = bp_df['likelihood']
    cos = np.cos(angles)
    sin = np.sin(angles)
    for axis in bp_df.columns:
        if axis == 'x':
            rotated.loc[:, (bodypart, axis)] = (bp_x * cos) + (bp_y * sin)
        elif axis == 'y':
            rotated.loc[:, (bodypart, axis)] = (bp_y * cos) - (bp_x * sin)
        elif axis == 'likelihood':
            rotated.loc[:, (bodypart, axis)] = bp_likelihood

rotated = rotated.drop(columns='likelihood', level=1)
df = rotated

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(whiten=True, svd_solver='full')

pca_transformed = pca.fit_transform(df)

expVar = pd.DataFrame(data=pca.explained_variance_ratio_, columns=['explained variance ratio'])
expVar
sns.barplot(data=expVar, y='explained variance ratio', x=list(range(1, len(expVar)+1)))

In [ ]:
pca_transformed_df = pd.DataFrame(data=pca_transformed, columns=list(range(1, len(expVar)+1)),
                               index=df.index)
pca_transformed_df = pca_transformed_df.iloc[:, :11]

In [ ]:
# index = np.arange(1000, 12001, 1000)
# n=index
# timing = pd.DataFrame(data=n, columns=['n'], index=index)

# from sklearn.manifold import TSNE
# import time

# tsne = TSNE(n_components=2, perplexity=50, n_iter=20000, n_iter_without_progress=500, init='random', n_jobs=-1, random_state=1)

# for i in tqdm(timing.index):
#     subset = pca_transformed_df.sample(timing.loc[i, 'n'])
#     start_time = time.time()
#     transformed = tsne.fit_transform(subset)
#     timing.loc[i, 'fit_transform'] = (time.time() - start_time)

In [ ]:
# sns.lineplot(data=timing, x='n', y='fit_transform')

In [ ]:
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.preprocessing import PolynomialFeatures

# polynomial_features= PolynomialFeatures(degree=2)
# x_poly = polynomial_features.fit_transform(timing['n'][:, np.newaxis])
# y = timing['fit_transform'][:, np.newaxis]

# model = LinearRegression()
# model.fit(x_poly, y)
# y_poly_pred = model.predict(x_poly)

# rmse = np.sqrt(mean_squared_error(y,y_poly_pred))
# r2 = r2_score(y,y_poly_pred)
# print(rmse)
# print(r2)

# newpred = np.arange(1000000, 15000001, 500000)
# newpred_poly = polynomial_features.fit_transform(newpred[:, np.newaxis])
# new_y_pred = model.predict(newpred_poly)

# plt.rcParams.update({'font.size': 15})

# plt.figure(figsize=(14,6))
# plt.subplot(1, 2, 1)
# plt.scatter(timing['n'], timing['fit_transform']/(60*60))
# plt.plot(timing['n'], y_poly_pred/(60*60),'r--')
# plt.legend(['r\u00b2= ' + str(round(r2, 3))])
# plt.xlabel('Number of datapoints')
# plt.ylabel('Embedding time (h/16 cores)')
# plt.subplot(1, 2, 2)
# plt.plot(newpred, new_y_pred/(60*60), 'r--')
# plt.xlabel('Number of datapoints')
# plt.ylabel("'Estimated' embedding time (h/16 cores)")
# plt.tight_layout()
# plt.savefig(r"C:\Users\serce\Desktop\tsne_timing.pdf")
# print("'Estimated' computation time on 15 million datapoints is",
#       (model.predict(polynomial_features.fit_transform(np.array([[15000000]]))))/(60*60),
#      'hours on 16 cores')


In [ ]:
# newpred = np.arange(20000, 250000, 1000)
# newpred_poly = polynomial_features.fit_transform(newpred[:, np.newaxis])
# new_y_pred = model.predict(newpred_poly)

# print(model.coef_)
# plt.figure()
# plt.plot(newpred, new_y_pred/(60*60))
# plt.xlabel('Number of datapoints')
# plt.ylabel('Estimated computation time (h/16 cores)')

'Estimated' computation time on 15 million datapoints is [[38041.84420021]] hours on 16 cores

'Estimated' computation time on 200k datapoints is ~10 hours on 16 cores

In [ ]:
import umap
import time

reducer = umap.UMAP(n_neighbors=100, verbose=True, n_epochs=1000, min_dist=0)
start_time = time.time()
transformed = reducer.fit_transform(df)
end_time = time.time() - start_time
print('UMAP fit_transform took {} seconds.'.format(end_time))

In [ ]:
umap_df = pd.DataFrame(data=transformed, index=df.index, columns=['UMAP1', 'UMAP2'])

In [ ]:
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', size=0.00001, legend=None)

In [ ]:
from sklearn.neighbors import KernelDensity

kde = KernelDensity(kernel='gaussian', bandwidth=1.5)
start_time = time.time()
kde.fit(transformed)
print("fitted in --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
scores = kde.score_samples(transformed)
print("scored in --- %s seconds ---" % (time.time() - start_time))
transformed['score'] = np.exp(scores)

In [ ]:
sns.kde